In [ ]:
# # =========================
# # 3. Duyệt file + lấy nhãn
# # =========================
# all_files = []
# all_labels = []

# # In ra các thư mục gốc mà os.walk sẽ đi qua
# print(f"Bắt đầu duyệt từ: {DATA_DIR}")

# for root, _, files in os.walk(DATA_DIR):
#     print(f"Đang duyệt trong thư mục: {root}") # Thêm dòng này để kiểm tra
#     for f in files:
#         if f.endswith(".wav"):
#             path = os.path.join(root, f)
#             try:
#                 code = f.split("-")[2]
#                 label = EMOTION_MAP[code]
#             except IndexError: # Thay đổi except chung thành IndexError cụ thể hơn
#                 print(f"File không chuẩn hoặc tên không đúng định dạng: {f}")
#                 continue
#             except KeyError: # Nếu code không có trong EMOTION_MAP
#                 print(f"Mã cảm xúc '{code}' không có trong EMOTION_MAP cho file: {f}")
#                 continue

#             # Chỉ để kiểm tra 1 vài file đầu tiên để không quá dài
#             if len(all_files) < 10: # Chỉ in 10 file đầu tiên
#                 print(f"  Thêm file: {path}")
#             all_files.append(path)
#             all_labels.append(label)

# print(f"Tổng số file sau khi duyệt: {len(all_files)}")

In [1]:
# =========================
# 1_preprocess_emotionid.py
# =========================
import os
import librosa
import soundfile as sf
import pandas as pd
from sklearn.model_selection import train_test_split

# =========================
# 1. Cấu hình
# =========================
DATA_DIR = "/kaggle/input/ravdess-emotional-speech-audio"  # dataset gốc
OUTPUT_DIR = "/kaggle/working/processed"  # thư mục lưu WAV + CSV
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Mapping code -> nhãn text
EMOTION_MAP = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

# Mapping EmotionID -> label_id theo gốc (0-index)
EMOTION_ID_TO_NUM = {k: int(k)-1 for k in EMOTION_MAP.keys()}
# '01' -> 0, '02' -> 1, ..., '08' -> 7

# =========================
# 2. Hàm preprocess audio
# =========================
def preprocess_audio(file_path, target_sr=16000):
    try:
        y, sr = librosa.load(file_path, sr=None)
        if y.ndim > 1:
            y = librosa.to_mono(y)
        y_resampled = librosa.resample(y, orig_sr=sr, target_sr=target_sr)
        return y_resampled, target_sr
    except Exception as e:
        print(f"Lỗi khi xử lý {file_path}: {e}")
        return None, None

# =========================
# 3. Duyệt file + lấy nhãn
# =========================
all_files = []
all_labels = []
all_label_ids = []

for root, _, files in os.walk(DATA_DIR):
    for f in files:
        if f.endswith(".wav"):
            path = os.path.join(root, f)
            try:
                code = f.split("-")[2]           # EmotionID từ filename
                label_text = EMOTION_MAP[code]   # nhãn text
                label_id = EMOTION_ID_TO_NUM[code]  # label_id theo gốc
            except:
                print(f"File không chuẩn: {f}")
                continue
            all_files.append(path)
            all_labels.append(label_text)
            all_label_ids.append(label_id)

print(f"Tổng số file: {len(all_files)}")

# =========================
# 4. Chia train/test
# =========================
train_files, test_files, train_labels, test_labels, train_label_ids, test_label_ids = train_test_split(
    all_files, all_labels, all_label_ids,
    test_size=0.2, stratify=all_label_ids, random_state=42
)

# =========================
# 5. Hàm lưu WAV + trả paths
# =========================
def save_wav(files, prefix):
    paths = []
    out_dir = os.path.join(OUTPUT_DIR, prefix)
    os.makedirs(out_dir, exist_ok=True)
    for i, path in enumerate(files):
        y, sr = preprocess_audio(path)
        if y is None:
            continue
        out_name = f"{prefix}_{i}.wav"
        out_path = os.path.join(out_dir, out_name)
        sf.write(out_path, y, sr)
        paths.append(out_path)
    return paths

train_paths = save_wav(train_files, "train")
test_paths = save_wav(test_files, "test")

# =========================
# 6. Lưu CSV final
# =========================
train_df = pd.DataFrame({
    "file_path": train_paths,
    "label": train_labels,
    "label_id": train_label_ids
})

test_df = pd.DataFrame({
    "file_path": test_paths,
    "label": test_labels,
    "label_id": test_label_ids
})

# Chia train -> validation
train_df, val_df = train_test_split(
    train_df, test_size=0.2, stratify=train_df["label_id"], random_state=42
)

# Lưu CSV
train_df.to_csv(os.path.join(OUTPUT_DIR, "train_final.csv"), index=False)
val_df.to_csv(os.path.join(OUTPUT_DIR, "val_final.csv"), index=False)
test_df.to_csv(os.path.join(OUTPUT_DIR, "test_final.csv"), index=False)

print(f"Số file train: {len(train_df)}, val: {len(val_df)}, test: {len(test_df)}")
print("✅ Hoàn tất preprocess, CSV sẵn sàng train mô hình")
print("Bảng label -> label_id:", dict(zip(EMOTION_MAP.values(), EMOTION_ID_TO_NUM.values())))


Tổng số file: 2880
Số file train: 1843, val: 461, test: 576
✅ Hoàn tất preprocess, CSV sẵn sàng train mô hình
Bảng label -> label_id: {'neutral': 0, 'calm': 1, 'happy': 2, 'sad': 3, 'angry': 4, 'fearful': 5, 'disgust': 6, 'surprised': 7}


In [ ]:
# 71%
# =========================
# 7) Tham số trích xuất & train
# =========================
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, utils
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import itertools, warnings, librosa, os, pandas as pd

warnings.filterwarnings("ignore", category=UserWarning)

SR_FEAT = 16000 

NAME_TO_ID_MAP = {name: int(code)-1 for code, name in EMOTION_MAP.items()}
ID_TO_NAME_MAP = {v: k for k, v in NAME_TO_ID_MAP.items()}

# --- THAM SỐ ---
N_MELS = 128
MAX_PAD_LEN = 200  # ~3.2 giây
BATCH_SIZE = 32  # Tăng lại để stable hơn
EPOCHS = 100
NUM_CLASSES = len(NAME_TO_ID_MAP)
INPUT_SHAPE = (N_MELS, MAX_PAD_LEN, 1)  # CHỈ DÙNG LOG-MEL, BỎ DELTA!

MODEL_OUT = os.path.join(OUTPUT_DIR, "ravdess_ser_cnn_simple.h5")

# =========================
# 8) Trích xuất features ĐơN GIẢN
# =========================

def add_white_noise(y, noise_factor=0.002):
    """Thêm nhiễu nhẹ."""
    noise = np.random.randn(len(y))
    return y + noise_factor * noise

def pitch_shift(y, sr=SR_FEAT, n_steps=2):
    """Thay đổi cao độ."""
    n = np.random.uniform(-n_steps, n_steps)
    return librosa.effects.pitch_shift(y, sr=sr, n_steps=n)

def pad_or_truncate(feat, max_len=MAX_PAD_LEN):
    """Pad hoặc cắt về max_len."""
    if feat.shape[1] < max_len:
        pad_width = max_len - feat.shape[1]
        feat = np.pad(feat, pad_width=((0, 0), (0, pad_width), (0, 0)), mode='constant')
    else:
        feat = feat[:, :max_len, :]
    return feat

def extract_features(y, sr=SR_FEAT, n_mels=N_MELS, max_pad_len=MAX_PAD_LEN):
    """
    CHỈ DÙNG LOG-MEL SPECTROGRAM - ĐƠN GIẢN NHẤT!
    """
    try:
        # Tính mel spectrogram
        melspec = librosa.feature.melspectrogram(
            y=y, 
            sr=sr, 
            n_mels=n_mels,
            n_fft=2048,
            hop_length=512,
            fmax=8000
        )
        
        # Chuyển sang dB scale
        log_melspec = librosa.power_to_db(melspec, ref=np.max)
        
        # Normalize về [-1, 1]
        log_melspec = (log_melspec - log_melspec.mean()) / (log_melspec.std() + 1e-8)
        
        # Thêm channel dimension
        log_melspec = np.expand_dims(log_melspec, axis=-1)
        
        # Pad/truncate
        log_melspec = pad_or_truncate(log_melspec, max_pad_len)
        
        return log_melspec
    
    except Exception as e:
        print(f"Error: {e}")
        return None

# =========================
# 9) Data Generator 
# =========================
class AudioDataGenerator(utils.Sequence):
    
    def __init__(self, df, batch_size, input_shape, num_classes, 
                 sr, n_mels, max_pad_len, shuffle=True, augment=False):
        self.df = df
        self.paths = df['file_path'].values
        self.labels = df['label_id'].values 
        
        self.batch_size = batch_size
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.sr = sr
        self.n_mels = n_mels
        self.max_pad_len = max_pad_len
        self.shuffle = shuffle
        self.augment = augment
        
        self.indexes = np.arange(len(self.paths))
        if self.shuffle:
            self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_paths = [self.paths[k] for k in batch_indexes]
        batch_labels = [self.labels[k] for k in batch_indexes]

        X = np.empty((self.batch_size, *self.input_shape))
        y = np.empty((self.batch_size), dtype=int)

        for i, file_path in enumerate(batch_paths):
            try:
                # Load audio
                y_audio, _ = librosa.load(file_path, sr=self.sr)
                
                # Augmentation ÍT HƠN
                if self.augment and np.random.rand() < 0.5:
                    if np.random.rand() < 0.5:
                        y_audio = add_white_noise(y_audio)
                    else:
                        y_audio = pitch_shift(y_audio, sr=self.sr)
                
                # Extract features
                feat = extract_features(
                    y_audio,
                    sr=self.sr, 
                    n_mels=self.n_mels, 
                    max_pad_len=self.max_pad_len
                )
                
                if feat is None:
                    feat = np.zeros(self.input_shape) 
                    
                X[i,] = feat
                y[i] = batch_labels[i]
                
            except Exception as e:
                print(f"Error: {e}")
                X[i,] = np.zeros(self.input_shape)
                y[i] = 0

        return X, utils.to_categorical(y, num_classes=self.num_classes)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

# =========================
# 10) Model ĐƠN GIẢN NHƯNG HIỆU QUẢ
# =========================
def build_cnn_model(input_shape, num_classes=NUM_CLASSES):
    """
    Kiến trúc tập trung vào generalization.
    """
    
    inp = layers.Input(shape=input_shape)
    
    # Block 1
    x = layers.Conv2D(32, (3, 3), padding='same')(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(32, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Block 2
    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Block 3
    x = layers.Conv2D(128, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(128, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Global pooling
    x = layers.GlobalAveragePooling2D()(x)
    
    # Dense layers - ÍT HƠN
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    # Output
    out = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inp, out)
    
    # Learning rate cao hơn với decay
    initial_lr = 0.001 
    optimizer = tf.keras.optimizers.Adam(learning_rate=initial_lr)
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# =========================
# 11) Plot functions
# =========================
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix'):
    if normalize:
        cm = cm.astype(float) / (cm.sum(axis=1, keepdims=True) + 1e-8)
    plt.figure(figsize=(10,8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title, fontsize=14)
    plt.colorbar()
    ticks = np.arange(len(classes))
    plt.xticks(ticks, classes, rotation=45, ha='right')
    plt.yticks(ticks, classes)
    thresh = cm.max() / 2.0
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        val = f"{cm[i, j]:.2f}" if normalize else f"{int(cm[i, j])}"
        plt.text(j, i, val, ha="center", va="center",
                 color="white" if cm[i, j] > thresh else "black", fontsize=10)
    plt.ylabel('True label', fontsize=12)
    plt.xlabel('Predicted label', fontsize=12)
    plt.tight_layout()
    plt.show()

def plot_history(history):
    plt.figure(figsize=(14,5))
    
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='Train Loss', linewidth=2)
    plt.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.legend(fontsize=10)
    plt.title('Training and Validation Loss', fontsize=14)
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='Train Acc', linewidth=2)
    plt.plot(history.history['val_accuracy'], label='Val Acc', linewidth=2)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.legend(fontsize=10)
    plt.title('Training and Validation Accuracy', fontsize=14)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# =========================
# 12) Training với COSINE ANNEALING
# =========================
def run_training():
    """Training với cosine annealing schedule."""
    
    def _filter_existing(df):
        df = df.copy()
        df["exists"] = df["file_path"].apply(lambda p: isinstance(p, str) and os.path.exists(p))
        df = df[df["exists"]].drop(columns=["exists"])
        return df

    # Load data
    train_df = _filter_existing(pd.read_csv(os.path.join(OUTPUT_DIR, "train_final.csv")))
    val_df   = _filter_existing(pd.read_csv(os.path.join(OUTPUT_DIR, "val_final.csv")))
    test_df  = _filter_existing(pd.read_csv(os.path.join(OUTPUT_DIR, "test_final.csv")))

    print("="*60)
    print(f"📊 Dataset:")
    print(f"   Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")
    print(f"📐 Config:")
    print(f"   Input: {INPUT_SHAPE}")
    print(f"   Batch: {BATCH_SIZE}")
    print("="*60)
    1
    # Create generators
    train_gen = AudioDataGenerator(
        train_df, BATCH_SIZE, INPUT_SHAPE, NUM_CLASSES, 
        SR_FEAT, N_MELS, MAX_PAD_LEN, shuffle=True, augment=True
    )
    val_gen = AudioDataGenerator(
        val_df, BATCH_SIZE, INPUT_SHAPE, NUM_CLASSES, 
        SR_FEAT, N_MELS, MAX_PAD_LEN, shuffle=False, augment=False
    )
    test_gen = AudioDataGenerator(
        test_df, BATCH_SIZE, INPUT_SHAPE, NUM_CLASSES, 
        SR_FEAT, N_MELS, MAX_PAD_LEN, shuffle=False, augment=False
    )
    
    # Build model
    print("\n  Building model...")
    model = build_cnn_model(input_shape=INPUT_SHAPE)
    model.summary()
    
    # Cosine annealing schedule
    def cosine_annealing(epoch, lr):
        """Cosine annealing schedule."""
        initial_lr = 0.001
        min_lr = 1e-6
        T_max = 30  # Restart period
        
        cycle = np.floor(1 + epoch / (2 * T_max))
        x = np.abs(epoch / T_max - 2 * cycle + 1)
        new_lr = min_lr + (initial_lr - min_lr) * (1 + np.cos(np.pi * x)) / 2
        return new_lr
    
    # Callbacks
    cbs = [
        callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=20,  # Tăng patience
            restore_best_weights=True,
            mode='max',
            verbose=1
        ),
        callbacks.LearningRateScheduler(cosine_annealing, verbose=1),
        callbacks.ModelCheckpoint(
            MODEL_OUT,
            save_best_only=True,
            monitor='val_accuracy',
            mode='max',
            verbose=1
        ),
    ]

    # Train
    print("\n Training...")
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=EPOCHS,
        callbacks=cbs,
        verbose=1
    )

    # Evaluate
    print("\n" + "="*60)
    print(" Testing...")
    model.load_weights(MODEL_OUT)
    
    loss, acc = model.evaluate(test_gen, verbose=0)
    print(f"\n Test Results:")
    print(f"   Loss:     {loss:.4f}")
    print(f"   Accuracy: {acc*100:.2f}%")
    print("="*60)

    # Predictions
    print("\n Generating predictions...")
    y_true, y_pred = [], []
    for i in range(len(test_gen)):
        Xb, yb = test_gen[i]
        pb = model.predict(Xb, verbose=0)
        y_pred.extend(np.argmax(pb, axis=1)) 
        y_true.extend(np.argmax(yb, axis=1))

    target_names = [ID_TO_NAME_MAP[i] for i in range(NUM_CLASSES)]
    
    print("\n" + "="*60)
    print(" Classification Report:")
    print("="*60)
    print(classification_report(y_true, y_pred, target_names=target_names, digits=4))
    print("="*60)

    # Confusion matrices
    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, classes=target_names, normalize=False, 
                         title="Confusion Matrix (Counts)")
    plot_confusion_matrix(cm, classes=target_names, normalize=True, 
                         title="Confusion Matrix (Normalized)")
    
    plot_history(history)
    
    print(f"\n Done! Model saved: {MODEL_OUT}")

if __name__ == "__main__":
    run_training()